In [8]:
import pandas as pd
import numpy as np
import re

In [9]:
import xmlrpc.client
from datetime import date, timedelta, datetime
import pandas as pd

# Conexión con Odoo (manteniendo tus credenciales)
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
weeks = 52

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=weeks)).strftime("%Y-%m-%d")

# --- 1) Buscar facturas válidas (account.invoice) ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),    # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas (account.invoice.line) ---

# Campos de la LÍNEA de factura. Eliminamos 'number', 'user_id', 'section_id', 'partner_id' porque irán en la factura.
line_fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","origin"] 

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": line_fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros de líneas...")

# --- 3) Pasar a DataFrame de líneas ---
line_df = pd.DataFrame(records).fillna(0)


# Separar product_id
line_df["product_id_num"] = line_df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["product_name"] = line_df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
line_df["invoice_id_num"] = line_df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["invoice_name"] = line_df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Convertir fecha a datetime
line_df["date_invoice"] = pd.to_datetime(line_df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
line_df = line_df.drop(columns=["invoice_id","create_date"])

print(f"Total de líneas descargadas: {len(line_df)}")

# ----------------------------------------------------
# --- 4) Descargar los campos adicionales de Factura (account.invoice) ---
# ----------------------------------------------------
# Añadimos los campos que quieres: number, user_id, section_id, y también partner_id y store_id
invoice_fields = ["id", "store_id", "number", "user_id", "section_id", "partner_id"]
invoices = models.execute_kw(
    db, uid, password,
    "account.invoice", "read",
    [invoice_ids], # Solo las facturas que encontramos
    {"fields": invoice_fields}
)
invoice_df = pd.DataFrame(invoices)

# --- 5) Procesar campos de la factura ---

# Separar store_id
invoice_df["store_name"] = invoice_df["store_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar user_id (Vendedor)
invoice_df["salesperson_name"] = invoice_df["user_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar section_id (Equipo de Ventas)
invoice_df["sales_team_name"] = invoice_df["section_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar partner_id (Cliente/Partner)
invoice_df["partner_id_num"] = invoice_df["partner_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
# El nombre del partner es el segundo elemento de la tupla (si existe)
invoice_df["client_name_inv"] = invoice_df["partner_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)


# Eliminar columnas originales no deseadas o ya procesadas
invoice_df = invoice_df.drop(columns=["store_id", "user_id", "section_id", "partner_id"])


# ----------------------------------------------------
# --- 6) Fusionar DataFrames ---
# ----------------------------------------------------

# Fusionamos las líneas de factura (line_df) con los datos de las facturas (invoice_df)
df = line_df.merge(
    invoice_df, 
    left_on="invoice_id_num", 
    right_on="id", 
    how="left"
)

# Limpieza final de columnas de IDs de factura
df = df.drop(columns=["invoice_id_num", "product_id_num"])


df['origin'] = df['origin'].astype('string')
df["product_id"] = df["product_id"].astype(str)

#6 MINUTOS

Facturas encontradas: 51392
Descargados 20000 registros de líneas...
Descargados 40000 registros de líneas...
Descargados 60000 registros de líneas...
Descargados 80000 registros de líneas...
Descargados 100000 registros de líneas...
Descargados 120000 registros de líneas...
Descargados 140000 registros de líneas...
Descargados 160000 registros de líneas...
Descargados 180000 registros de líneas...
Descargados 200000 registros de líneas...
Descargados 200773 registros de líneas...
Total de líneas descargadas: 200773


In [15]:
# Referncia de producto
df["product_ref"] = df["product_name"].str.extract(r"\[([A-Z0-9]+)\]")
df['ref_prod'] = df['product_name'].apply(lambda x: re.search(r'\(([^)]+)\)$', x).group(1) if pd.notna(x) and re.search(r'\(([^)]+)\)$', x) else None)
df['donsson'] = df['product_ref'].str.endswith('025')


df_d = df[df["donsson"]==True]

df_d = df_d.copy()


df_d["date"] = pd.to_datetime(df_d["date_invoice"])
df_d["año"] = df_d["date"].dt.year
df_d["mes"] = df_d["date"].dt.month


df_pivot = df_d.copy()
df_pivot = pd.pivot_table(df_d , values="quantity", index=["product_name","ref_prod","product_ref"] , columns=["año","mes"] , aggfunc= np.sum , fill_value= 0)
df_pivot['Total'] = df_pivot.sum(axis=1)
df_pivot['Promedio'] = df_pivot.mean(axis=1)


# 1. Sumar a nivel de línea para obtener el total
df_pivot['Total'] = df_pivot.iloc[:, 1:].sum(axis=1)

# 2. Ordenar el DataFrame por el total de ventas de forma descendente
df_pivot = df_pivot.sort_values(by='Total', ascending=False)

# 3. Calcular el porcentaje acumulado
df_pivot['acum%'] = df_pivot['Total'].cumsum() / df_pivot['Total'].sum()

# 4. Asignar la clasificación de Pareto
def clasificar_pareto(valor):
    if valor < 0.5:
        return "AAA"
    elif valor < 0.8:
        return "A"
    
    elif valor < 0.95:
        return "B"

    else:
        return "C"

df_pivot["Clasificacion"] = df_pivot["acum%"].apply(clasificar_pareto)

df_pivot = df_pivot.reset_index()

/tmp/ipykernel_92486/1169317552.py:18: FutureWarning: The provided callable <function sum at 0x7b2938505800> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_pivot = pd.pivot_table(df_d , values="quantity", index=["product_name","ref_prod","product_ref"] , columns=["año","mes"] , aggfunc= np.sum , fill_value= 0)


In [16]:
df_pivot.head(10)

año                                       product_name    ref_prod  \
mes                                                                  
0    [DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...  025 DA2570   
1    [DAB02788025] DA2788  FILTRO AIRE DONSSON - Ch...  025 DA2788   
2    [DAB02772025] DA2772 FILTRO AIRE BOBCAT, HITAC...  025 DA2772   
3    [DAB04570025] DA4570 FILTRO AIRE 2. DONSSON - ...  025 DA4570   
4    [DAB02666025] DA2666 FILTRO AIRE 1_ DONSSON -B...  025 DA2666   
5    [DAB02671025] DA2671 FILTROAIRE KUBOTA, NISSAN...  025 DA2671   
6    [DAB02968025] DA2968 FILTRO AIRE- HINO  (025 D...  025 DA2968   
7    [DAB08168025] DA8168 FILTRO AIRE DONSSON - FOT...  025 DA8168   
8    [DAB02982025] DA2982 FILTRO AIRE DONSSON -  IH...  025 DA2982   
9    [DAB04666025] DA4666 FILTRO AIRE 2o. DONSSON B...  025 DA4666   

año  product_ref    2024                   2025                          \
mes                   10     11      12       1       2       3       4   
0    DAB02570025  1452.0  966.0  1019.0  1365.0  1351.0  1485.0  1616.0   
1    DAB02788025   689.0  793.0  1034.0   755.0   855.0   884.0  1301.0   
2    DAB02772025   703.0  492.0   973.0   660.0   564.0  1206.0  1093.0   
3    DAB04570025   962.0  961.0   858.0  1107.0  1422.0  1131.0   961.0   
4    DAB02666025   578.0  386.0   560.0   855.0   793.0   831.0  1069.0   
5    DAB02671025   474.0  439.0   690.0   513.0   425.0   638.0   351.0   
6    DAB02968025   277.0  503.0   596.0   299.0   475.0   388.0   357.0   
7    DAB08168025   135.0  334.0   313.0   272.0   406.0   346.0   474.0   
8    DAB02982025   197.0  387.0   259.0   376.0   350.0   342.0   313.0   
9    DAB04666025   122.0  326.0   397.0   592.0   391.0   424.0   633.0   

año                                                        Total     Promedio  \
mes       5       6       7       8       9     10                              
0    1847.0  1106.0  1094.0  2083.0  1092.0  578.0  35092.285714  2436.285714   
1    1048.0   982.0  1051.0   845.0  1220.0  681.0  25321.000000  1734.000000   
2    1098.0   799.0   696.0  2419.0   714.0  299.0  24402.714286  1673.714286   
3    1335.0   728.0   685.0   144.0    42.0    3.0  21193.000000  1477.000000   
4     952.0   551.0   674.0  1019.0   512.0  238.0  18746.285714  1288.285714   
5    1007.0   513.0   362.0   663.0   401.0  164.0  13754.571429   948.571429   
6     493.0   294.0   455.0   456.0   554.0  135.0  11041.571429   754.571429   
7     421.0   372.0   347.0   453.0   462.0  333.0   9867.857143   666.857143   
8     471.0   355.0   460.0   325.0   468.0  228.0   9512.285714   647.285714   
9     439.0   341.0   192.0   173.0    42.0    2.0   8608.000000   582.000000   

año     acum% Clasificacion  
mes                          
0    0.059031           AAA  
1    0.101625           AAA  
2    0.142674           AAA  
3    0.178324           AAA  
4    0.209858           AAA  
5    0.232995           AAA  
6    0.251569           AAA  
7    0.268168           AAA  
8    0.284169           AAA  
9    0.298649           AAA

In [17]:
df_pivot.groupby("Clasificacion")["Clasificacion"].count()

Clasificacion
A      101
AAA     30
B      183
C      422
Name: Clasificacion, dtype: int64

In [18]:
df_pivot[df_pivot["ref_prod"]=="025 DA8137"]

año                                       product_name    ref_prod  \
mes                                                                  
227  [DAB08137025] DA8137 FILTRO AIRE KENWORTH 15" ...  025 DA8137   

año  product_ref 2024              2025                                   \
mes                10    11    12     1    2     3     4    5    6     7   
227  DAB08137025  4.0  31.0  19.0  16.0  6.0  18.0  18.0  4.0  8.0  26.0   

año                         Total   Promedio     acum% Clasificacion  
mes     8     9    10                                                 
227  24.0  13.0  13.0  424.571429  28.571429  0.903296             B

In [19]:
df_pivot.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx")